In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import pyarrow as pa
import pyarrow.parquet as pq
import warnings
warnings.filterwarnings("ignore")

In [16]:

df_g = pd.read_csv(r"C:\Users\Admin\Downloads\Primer Proyecto\DS\steam_games_clean.csv")
df_r = pd.read_csv(r"C:\Users\Admin\Downloads\Primer Proyecto\DS\user_reviews_clean.csv")
df_i = pd.read_csv(r"C:\Users\Admin\Downloads\Primer Proyecto\DS\user_items_clean.csv")


,user_id,item_id,item_count,item_name,playtime_forever
0,76561197970982479,10,277,counter-strike,6
1,76561197970982479,20,277,team fortress classic,0
2,76561197970982479,30,277,day of defeat,7
3,76561197970982479,40,277,deathmatch classic,0
4,76561197970982479,50,277,half-life: opposing force,0
...,...,...,...,...,...
5094087,76561198329548331,346330,7,brainbread 2,0
5094088,76561198329548331,373330,7,all is dust,0
5094089,76561198329548331,388490,7,one way to die: steam edition,3
5094090,76561198329548331,521570,7,you have 10 seconds 2,4


In [75]:
#Columnas necesarios del DF Steam_Games
df_filtrado = df_g[['price', 'release_year', 'developer', 'id']]
# se eliminan los duplicados
df_developer = df_filtrado.drop_duplicates()


# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_developer)
# Define las opciones de compresión
compression = 'None'
# Guarda el objeto Table en formato Parquet con compresión Snappy
pq.write_table(table, r"C:\Users\Admin\Downloads\Primer Proyecto\PI_ML_OPS-FT\Dataset\developer.parquet", compression=compression)
df_developer

,price,release_year,developer,id
0,4.99,2018,kotoshiro,761140
5,0.00,2018,secret level srl,643980
9,0.00,2017,poolians.com,670290
14,0.99,2017,彼岸领域,767400
17,3.99,2018,trickjump games ltd,772540
...,...,...,...,...
71535,1.99,2018,bidoniera games,745400
71539,1.99,2018,"nikita ""ghost_rus""",773640
71543,4.99,2018,sacada,733530
71546,1.99,2018,laush dmitriy sergeevich,610660


In [28]:
#User_data
#Dinero Gastado por usuario
dinero_gastado = df_g[['price', 'id']]
# Renombrar la columna 'id' a 'item_id' para mergear mas tarde
dinero_gastado = dinero_gastado.rename(columns={'id': 'item_id'})
dinero_gastado = dinero_gastado.drop_duplicates(subset='item_id', keep='first')
dinero_gastado



,price,item_id
0,4.99,761140
5,0.00,643980
9,0.00,670290
14,0.99,767400
17,3.99,772540
...,...,...
71535,1.99,745400
71539,1.99,773640
71543,4.99,733530
71546,1.99,610660


In [29]:
#Cantidad de items
cantidad_items = df_i[['item_count', 'user_id', 'item_id']]
cantidad_items

,item_count,user_id,item_id
0,277,76561197970982479,10
1,277,76561197970982479,20
2,277,76561197970982479,30
3,277,76561197970982479,40
4,277,76561197970982479,50
...,...,...,...
5094087,7,76561198329548331,346330
5094088,7,76561198329548331,373330
5094089,7,76561198329548331,388490
5094090,7,76561198329548331,521570


In [30]:
#Merge a ambos DF's 
df_user_data = cantidad_items.merge(dinero_gastado, on='item_id', how='left')
df_user_data

,item_count,user_id,item_id,price
0,277,76561197970982479,10,9.99
1,277,76561197970982479,20,4.99
2,277,76561197970982479,30,4.99
3,277,76561197970982479,40,4.99
4,277,76561197970982479,50,4.99
...,...,...,...,...
5094087,7,76561198329548331,346330,0.00
5094088,7,76561198329548331,373330,NaN
5094089,7,76561198329548331,388490,0.00
5094090,7,76561198329548331,521570,0.00


In [31]:
#Completar valores nulos con 0.0
df_user_data['price'] = df_user_data['price'].fillna(0.0)

In [35]:
#Sumar dinero gastado por usuario y guardar en una copia para otr merge
df_user_data_parcial = df_user_data.groupby('user_id').agg({'price': 'sum', 'item_count': 'first'}).reset_index()

# Cambiar el nombre de la columna 'precio' a 'dinero_gastado'
df_user_data_parcial = df_user_data_parcial.rename(columns={'price': 'total_spent'})
df_user_data_parcial

,user_id,total_spent,item_count
0,--000--,397.78,58
1,--ace--,166.82,44
2,--ionex--,99.93,23
3,-2SV-vuLB-Kg,427.50,68
4,-404PageNotFound-,1509.32,149
...,...,...,...
70907,zzonci,19.98,5
70908,zzoptimuszz,64.98,61
70909,zzydrax,99.94,13
70910,zzyfo,828.51,84


In [36]:
#Guardamos 3 columnas del DF reviews
recomienda_items = df_r[['recommend', 'user_id', 'item_id']]
recomienda_items

,recommend,user_id,item_id
0,True,76561197970982479,1250
1,True,76561197970982479,22200
2,True,76561197970982479,43110
3,True,js41637,251610
4,True,js41637,227300
...,...,...,...
59300,True,76561198312638244,70
59301,True,76561198312638244,362890
59302,True,LydiaMorley,273110
59303,True,LydiaMorley,730


In [37]:
# Calcular el porcentaje de recommend True por user_id
total_count = recomienda_items.groupby('user_id').size()
recommend_true_count = recomienda_items[recomienda_items['recommend'] == True].groupby('user_id').size() 

# Llenar con 0 los valores faltantes
recommend_true_count = recommend_true_count.reindex(total_count.index, fill_value=0)

# Calcular el porcentaje
percentage = (recommend_true_count / total_count) * 100

# Crear un nuevo dataframe con los resultados
recomendacion = pd.DataFrame({'user_id': percentage.index, 'recommend': percentage.values})
recomendacion

,user_id,recommend
0,--000--,100.0
1,--ace--,100.0
2,--ionex--,100.0
3,-2SV-vuLB-Kg,100.0
4,-Azsael-,100.0
...,...,...
25453,zwanzigdrei,100.0
25454,zy0705,100.0
25455,zynxgameth,100.0
25456,zyr0n1c,100.0


In [38]:
#Merge final de los DF Creados
df_user_data_final = df_user_data_parcial.merge(recomendacion, on='user_id', how='left')
df_user_data_final['recommend'].fillna(0, inplace=True)
df_user_data_final

,user_id,total_spent,item_count,recommend
0,--000--,397.78,58,100.0
1,--ace--,166.82,44,100.0
2,--ionex--,99.93,23,100.0
3,-2SV-vuLB-Kg,427.50,68,100.0
4,-404PageNotFound-,1509.32,149,0.0
...,...,...,...,...
70907,zzonci,19.98,5,0.0
70908,zzoptimuszz,64.98,61,100.0
70909,zzydrax,99.94,13,0.0
70910,zzyfo,828.51,84,0.0


In [41]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_user_data_final)

# Define las opciones de compresión con Snappy
compression = 'None'

pq.write_table(table, r"C:\Users\Admin\Downloads\Primer Proyecto\PI_ML_OPS-FT\Dataset\user_data.parquet", compression=compression)

Def User_for_genre

Usuario que acumula mas horas jugadas por género y lista de acumulacion de horas jugadas por año de lanzamiento.

In [43]:
#Obtenemos columna tiempo jugado, user_ID, item_id
df_user = df_i[['playtime_forever', 'user_id', 'item_id']]
df_user

,playtime_forever,user_id,item_id
0,6,76561197970982479,10
1,0,76561197970982479,20
2,7,76561197970982479,30
3,0,76561197970982479,40
4,0,76561197970982479,50
...,...,...,...
5094087,0,76561198329548331,346330
5094088,0,76561198329548331,373330
5094089,3,76561198329548331,388490
5094090,4,76561198329548331,521570


In [46]:
df_genero = df_g[['release_year', 'genres', 'id']]
# Renombrar la columna 'id' a 'item_id' para mergearlas mas tarde
df_genero = df_genero.rename(columns={'id': 'item_id'})
df_genero

,release_year,genres,item_id
0,2018,Action,761140
1,2018,Casual,761140
2,2018,Indie,761140
3,2018,Simulation,761140
4,2018,Strategy,761140
...,...,...,...
71546,2018,Indie,610660
71547,2018,Racing,610660
71548,2018,Simulation,610660
71549,2017,Casual,658870


In [47]:
df_user_genre = df_genero.merge(df_user, on='item_id')
df_user_genre

,release_year,genres,item_id,playtime_forever,user_id
0,1997,Action,282010,5,UTNerd24
1,1997,Action,282010,0,I_DID_911_JUST_SAYING
2,1997,Action,282010,0,76561197962104795
3,1997,Action,282010,0,r3ap3r78
4,1997,Action,282010,13,saint556
...,...,...,...,...,...
9877276,2004,Action,80,0,76561198273508956
9877277,2004,Action,80,0,76561198282090798
9877278,2004,Action,80,0,943525
9877279,2004,Action,80,9,76561198283312749


In [70]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_user_genre)

# Define las opciones de compresión con Snappy
compression = 'None'

pq.write_table(table, r"C:\Users\Admin\Downloads\Primer Proyecto\PI_ML_OPS-FT\Dataset\user_for_genre.parquet", compression=compression)

DEF best_developer_year
TOP 3 desarrolladores con mas recomendaciones por año.

In [60]:
df_dev = df_g[['release_year', 'developer', 'id']]
df_dev = df_dev.rename(columns={'id': 'item_id'})
df_dev

,release_year,developer,item_id
0,2018,kotoshiro,761140
1,2018,kotoshiro,761140
2,2018,kotoshiro,761140
3,2018,kotoshiro,761140
4,2018,kotoshiro,761140
...,...,...,...
71546,2018,laush dmitriy sergeevich,610660
71547,2018,laush dmitriy sergeevich,610660
71548,2018,laush dmitriy sergeevich,610660
71549,2017,"xropi,stev3ns",658870


In [52]:
#Columnas necesarias del DF user_reviews
df_recomen = df_r[['recommend', 'user_id', 'item_id']]
df_recomen

,recommend,user_id,item_id
0,True,76561197970982479,1250
1,True,76561197970982479,22200
2,True,76561197970982479,43110
3,True,js41637,251610
4,True,js41637,227300
...,...,...,...
59300,True,76561198312638244,70
59301,True,76561198312638244,362890
59302,True,LydiaMorley,273110
59303,True,LydiaMorley,730


In [53]:
#Mergeamos los DF's 
df_best_developer = df_dev.merge(df_recomen, on='item_id')
df_best_developer

,release_year,genres,item_id,recommend,user_id
0,1997,Action,282010,True,InstigatorAU
1,1997,Indie,282010,True,InstigatorAU
2,1997,Racing,282010,True,InstigatorAU
3,1998,Action,70,True,EizanAratoFujimaki
4,1998,Action,70,True,GamerFag
...,...,...,...,...,...
126227,2004,Action,80,False,76561198023508728
126228,2004,Action,80,True,Lone_walker
126229,2004,Action,80,True,green290
126230,2004,Action,80,True,174gamecuman700kngkakak


In [61]:
# Elimino los registros con release_year = sin_dato_fecha 
df_best_developer.drop(df_best_developer[df_best_developer['release_year'] == "Sin_dato_fecha"].index, inplace=True)

# convierto la columna a entero, para poder utilizar bien la función 
df_best_developer['release_year'] = df_best_developer['release_year'].astype(int)

In [69]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_best_developer)

# Define las opciones de compresión con Snappy
compression = 'None'

pq.write_table(table, r"C:\Users\Admin\Downloads\Primer Proyecto\PI_ML_OPS-FT\Dataset\developer_of_year.parquet", compression=compression)

DEF developer_reviews_analysis
Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [63]:
#Columnas del df  reviews
df_analisis = df_r[['sentiment_analysis', 'item_id']]
df_analisis

,sentiment_analysis,item_id
0,1,1250
1,1,22200
2,1,43110
3,1,251610
4,1,227300
...,...,...
59300,2,70
59301,2,362890
59302,1,273110
59303,1,730


In [64]:
#Columnas del df Steam games
df_develop = df_g[['developer', 'id']]
df_develop = df_develop.rename(columns={'id': 'item_id'})
df_develop

,developer,item_id
0,kotoshiro,761140
1,kotoshiro,761140
2,kotoshiro,761140
3,kotoshiro,761140
4,kotoshiro,761140
...,...,...
71546,laush dmitriy sergeevich,610660
71547,laush dmitriy sergeevich,610660
71548,laush dmitriy sergeevich,610660
71549,"xropi,stev3ns",658870


In [65]:
#Mergeamos los dfs
df_developer_reviews = df_develop.merge(df_analisis, on='item_id')
df_developer_reviews

,developer,item_id,sentiment_analysis
0,stainless games ltd,282010,1
1,stainless games ltd,282010,1
2,stainless games ltd,282010,1
3,valve,70,1
4,valve,70,2
...,...,...,...
126227,valve,80,1
126228,valve,80,2
126229,valve,80,2
126230,valve,80,2


In [68]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_developer_reviews)

# Define las opciones de compresión con Snappy
compression = 'None'

pq.write_table(table, r"C:\Users\Admin\Downloads\Primer Proyecto\PI_ML_OPS-FT\Dataset\developer_review.parquet", compression=compression)

MODELO DE RECOMENDACION
Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

In [71]:
#Seleccionamos las columnas necesarias de cada DF
df_pre_modelo_g = df_g[['genres', 'title', 'id', 'developer']]
df_pre_modelo_g = df_pre_modelo_g.rename(columns={'id': 'item_id'})

df_pre_modelo_r = df_r[['item_id', 'recommend']]

#Mergeamos
df_modelo = df_pre_modelo_g.merge(df_pre_modelo_r, on='item_id')
df_modelo


,genres,title,item_id,developer,recommend
0,Action,carmageddon max pack,282010,stainless games ltd,True
1,Indie,carmageddon max pack,282010,stainless games ltd,True
2,Racing,carmageddon max pack,282010,stainless games ltd,True
3,Action,half-life,70,valve,True
4,Action,half-life,70,valve,True
...,...,...,...,...,...
126227,Action,counter-strike: condition zero,80,valve,False
126228,Action,counter-strike: condition zero,80,valve,True
126229,Action,counter-strike: condition zero,80,valve,True
126230,Action,counter-strike: condition zero,80,valve,True


In [72]:
#Trabajando el DF
# Calcular el porcentaje de recomendación agrupando por title
recommendation_percentage = (df_modelo.groupby('title')['recommend'].mean() * 100).round(1)

# Fusionar los resultados con el DataFrame original
df_filtrado = df_modelo.merge(recommendation_percentage, on='title')

# Eliminar los valores duplicados y quedarse con una fila por genres-title
df_modelo_recomendacion = df_filtrado.drop_duplicates(subset=['genres', 'title'])
df_modelo_recomendacion

,genres,title,item_id,developer,recommend_x,recommend_y
0,Action,carmageddon max pack,282010,stainless games ltd,True,100.0
1,Indie,carmageddon max pack,282010,stainless games ltd,True,100.0
2,Racing,carmageddon max pack,282010,stainless games ltd,True,100.0
3,Action,half-life,70,valve,True,98.4
65,Action,the ship: murder party,2400,outerlight ltd.,True,95.7
...,...,...,...,...,...,...
126204,Action,deathmatch classic,40,valve,True,100.0
126205,Action,half-life: blue shift,130,gearbox software,True,100.0
126210,Action,day of defeat,30,valve,True,75.0
126214,Action,unreal tournament 2004: editor's choice edition,13230,"epic games, inc.",True,100.0


In [73]:
# Eliminar la columna 'recommend_x'
df_modelo_recomendacion = df_modelo_recomendacion.drop('recommend_x', axis=1)

In [74]:
# Convierte el DataFrame a un objeto Table de PyArrow
table = pa.Table.from_pandas(df_modelo_recomendacion)

# Define las opciones de compresión con Snappy
compression = 'None'

# Guarda el objeto Table en formato Parquet con compresión Snappy
pq.write_table(table, r"C:\Users\Admin\Downloads\Primer Proyecto\PI_ML_OPS-FT\Dataset\recomendar_juego.parquet", compression=compression)
